In [38]:
import pandas as pd
import os
import requests

In [39]:
raidDir   = '/Users/amcguire/Documents/PERSONAL/KOL/raidLogs/'
newRDir   = '/Users/amcguire/Library/Application Support/KoLmafia/data/test/'
outputDir = '/Users/amcguire/Documents/PERSONAL/KOL/'

multiUser = {'madowl (combined)':['themadowl','madowl','Arkcon']}

clanMap   = {'The Hogs of Exploitery':'HoE', 
             'KirbyLlama\'s Private Dungeon Clan': 'KirbyClan',
             'The Piglets of Fate': 'Piglets',
             'The Hogs of Destiny': 'HoD',
             'The 100% of Scientific FACT Club': 'FactClub',
             'Castle Greyhawk': 'Greyhawk'}

finishedUsers = ['violetinsane','Captain Scotch','kenny kamAKAzi']

runFiles  = ['r1-3389','r2-3869','r3-4283','r4-4190','r5-3503','r6-3598',
             'r7-4372','r8-3473','r9-3858','r10-3787','r11-3797','r12-3798',
             'r13-12260','r14-8393','r15-3769','r16-3496','r17-4041','r18-3582',
             'r19-4318','r20-3584','r21-3794','r22-4698','r25-3802',
             'r69-1337']
             #'r24p-1852','r23p-722','r26p-900','r27p-2770']

In [53]:
# Temporary script to convert Stary script output into old c/p format

for file in os.listdir(newRDir):
    print(file)
    try:    
        clan = file.split(sep='_')[1]
        rNum = file.split(sep='_')[2][:6]
        newFileName = 'r{1}_{0}.txt'.format(clanMap[clan],rNum)

        currFile = open('{}/{}'.format(newRDir, file)).read()

        newFile = ''

        for ct, line in enumerate(currFile.split(sep='\n')):
            try:     newFile = newFile + line.split(sep='\t')[1].replace('<b>','').replace('</b>','') + '\n'
            except:  newFile = newFile + 'LINE-ERROR' + '\n'

        print('  - {} file parsed. {} new lines.'.format(clan,ct))

        file = open('{}{}'.format(raidDir,newFileName),"w")

        file.write(newFile)

        file.close()

        runFiles = runFiles + [newFileName[:-4]]
    except:
        print('  - !!! stupid thing fucked up !!!')

dungeon_The Hogs of Exploitery_189472.txt
  - The Hogs of Exploitery file parsed. 35 new lines.
.DS_Store
  - !!! stupid thing fucked up !!!
dungeon_The Hogs of Exploitery_188769.txt
  - The Hogs of Exploitery file parsed. 184 new lines.
dungeon_KirbyLlama's Private Dungeon Clan_188757.txt
  - KirbyLlama's Private Dungeon Clan file parsed. 189 new lines.
dungeon_The Hogs of Destiny_188819.txt
  - The Hogs of Destiny file parsed. 189 new lines.
dungeon_Castle Greyhawk_188871.txt
  - Castle Greyhawk file parsed. 119 new lines.
dungeon_The Piglets of Fate_188579.txt
  - The Piglets of Fate file parsed. 283 new lines.
dungeon_The Hogs of Destiny_188572.txt
  - The Hogs of Destiny file parsed. 139 new lines.
dungeon_The Piglets of Fate_189481.txt
  - The Piglets of Fate file parsed. 2 new lines.
dungeon_The Hogs of Exploitery_189236.txt
  - The Hogs of Exploitery file parsed. 166 new lines.
dungeon_The Piglets of Fate_189252.txt
  - The Piglets of Fate file parsed. 125 new lines.
dungeon_Ki

In [41]:
# Populate our run dictionary
allRuns = ""

for run, txtLog in enumerate(runFiles):
    # Python is zero-indexed, so we increment run by 1 here. 
    #   Also replace spaces w/ _ for playername.
    
    allRuns = allRuns+open('{}{}.txt'.format(raidDir, txtLog)).read()+"\n"

In [42]:
def collectRaiders(runs):
    """ Function to collect all dungeon raiders. Only grabs folks who
        have defeated at least one monster across all ASS runs. """
    
    raiders = []
    
    for row in runs.split('\n'):
        if ('defeated' in row) & (' (#' in row):
            name = row[:row.find(' (#')]
            raiders = list(set(raiders + [name]))
    
    return raiders
            
collectRaiders(allRuns)

['Just Eyes',
 'Stary',
 'RandomExtremity',
 'madowl',
 'John Keel',
 'The Dictator',
 'Arkcon',
 'gregmasta',
 'Whym',
 'Lyft',
 'Gausie',
 'HeroBill',
 'Volotani',
 'NukebOy1313',
 'Name Guy Man',
 'Phillammon',
 'kirByllAmA',
 'violetinsane',
 'AeNimUS',
 'playultm8',
 'threebullethamburgler',
 'ANonamUS',
 'themadowl',
 'tdog21',
 'Liora',
 'chameco',
 'Captain Scotch',
 'kenny kamAKAzi']

In [43]:
def tabulateKills(name, runs):
    """ Function to tabulate how many kills a specific individual had.
        Also outputs defeats, if we actually want that. """
    
    kills  = 0
    killed = 0
    
    for row in runs.split('\n'):
        if (name in row) & ('defeated' in row):
            if row[0:len(name)] == name: # fixing the madowl bug ;-;
                if 'was defeated' in row:
                    tSpent = int(row.split(' (')[2].split(' ')[0])
                    killed += tSpent
                else:
                    tSpent = int(row.split(' (')[2].split(' ')[0])
                    kills  += tSpent
                
    return({'kills':kills,'defeats':killed})

In [44]:
def tabulateLoots(name, runs):
    ''' Function to tabulate loot per individuals participating. Also 
        grabs skills, just so that we have em.'''
    
    loot   = []
    skills = 0
    
    for row in runs.split('\n'):
        if ('distributed' in row) and (name in row):
            distTo = row.split(' to ')[1].split(' (#')[0]
            if distTo == name:
                thisLoot = row.split(' distributed ')[1].split(' to ')[0]
                loot = loot + [thisLoot]
        elif (' used The Machine' in row) and (name in row):
            distTo = row.split(' used The ')[0].split(' (#')[0]
            if distTo == name: skills+=1
        
    # Parse out things we don't want to be considered as "true" loot
    notRealLoot = ['ghost pepper','electric Kool-Aid','skull capacitor',
                   'wriggling severed nose','Hunger™ Sauce','bottle of Bloodweiser']
    finalLoot = [x for x in loot if x not in notRealLoot]
    
    return({'loot':finalLoot,'skills':skills})
            
tabulateLoots('Captain Scotch', allRuns)

{'loot': ["Drunkula's wineglass",
  'zombie accordion',
  "Mayor Ghost's cloak",
  "Drunkula's bell",
  "Mayor Ghost's toupee",
  "Great Wolf's headband",
  "Mayor Ghost's sash",
  "Great Wolf's headband",
  "Unkillable Skeleton's restless leg",
  "Unkillable Skeleton's restless leg",
  'Quiets-Your-Steps',
  'Gets-You-Drunk',
  'HOA zombie eyes',
  'Gets-You-Drunk',
  "Drunkula's ring of haze",
  "Mayor Ghost's scissors",
  'Helps-You-Sleep'],
 'skills': 11}

In [45]:
finalDisplay = {}

# Create summary dictionary via tabulation code

for name in collectRaiders(allRuns):
    
    finalDisplay[name] = {}
    
    finalDisplay[name].update(tabulateKills(name, allRuns))
    finalDisplay[name].update(tabulateLoots(name, allRuns))
    finalDisplay[name]['finishedFlag'] = [1 if name in finishedUsers else 0][0]
    
    finalDisplay[name]['lootCount'] = len(finalDisplay[name]['loot'])

In [46]:
# Slice of code for combining duplicate users

for name in multiUser.keys():
    
    finalDisplay[name] = {}
    
    for field in ['kills','defeats','loot','skills','lootCount']:
        try:    finalDisplay[name][field] = sum([finalDisplay[x][field] for x in multiUser[name]])
        except: finalDisplay[name][field] = [].append([finalDisplay[x][field] for x in multiUser[name]])
            
    for x in multiUser[name]: finalDisplay.pop(x)

In [47]:
finalFrame = pd.DataFrame.from_dict(finalDisplay,orient='index').sort_values(by='kills',ascending=False).fillna(0)
finalFrame = finalFrame.loc[:,['kills','defeats','skills','lootCount','loot','finishedFlag']]
finalFrame['kill/skillz'] = finalFrame['kills']/(finalFrame['skills']+0.5)

finalFrame = finalFrame.sort_values(['finishedFlag','kill/skillz'], ascending=[True,False])
finalFrame

,kills,defeats,skills,lootCount,loot,finishedFlag,kill/skillz
Just Eyes,7493,92,5,7,"[Gets-You-Drunk, Drunkula's wineglass, Protect...",0.0,1362.363636
madowl (combined),14884,738,11,21,0,0.0,1294.260870
Name Guy Man,7569,7,6,17,"[Quiets-Your-Steps, Covers-Your-Head, Drunkula...",0.0,1164.461538
threebullethamburgler,6299,50,5,11,"[Drunkula's silky pants, Mayor Ghost's cloak, ...",0.0,1145.272727
Liora,548,7,0,1,[Great Wolf's left paw],0.0,1096.000000
gregmasta,13611,25,12,12,"[Gets-You-Drunk, Unkillable Skeleton's shingua...",0.0,1088.880000
Stary,6499,64,6,7,"[Mayor Ghost's scissors, Drunkula's cape, Grea...",0.0,999.846154
RandomExtremity,6452,30,6,7,"[Drunkula's bell, Great Wolf's rocket launcher...",0.0,992.615385
John Keel,4439,53,4,2,"[Unkillable Skeleton's breastplate, Unkillable...",0.0,986.444444
Whym,5368,1,5,1,[Gets-You-Drunk],0.0,976.000000


In [48]:
print("This effort has gotten {} skills for {} Loathers. Good job folks!".format(finalFrame['skills'].sum(),len(finalFrame['skills'])))

This effort has gotten 115 skills for 26 Loathers. Good job folks!


In [49]:
from datetime import date

currDate = str(date.today().strftime('%m%d'))
finalFrame.to_csv('{}dungeonLogs{}.csv'.format(outputDir,currDate))

In [224]:
def lineParse(string):
    ''' The buffer bit is when I was erroneously thinking that defeats somehow 
        counted against us. They do not. Still, left the logic in just in case
        I want to make this more complicated in the future, and get differential
        information for different parsing situations. Alas.'''
    
    if 'was defeated' in string:
        return {'buffer':int(string.split(' (')[2].split(' ')[0]),'kills':0}
    else:
        try:
            if ') defeated' not in string: print(string)
            return {'kills':int(string.split(' (')[2].split(' ')[0]),'buffer':0}
        except: 
            # print out when this encounters an error; this is how i found all the exceptions below
            print(string)
            return {'buffer':0,'kills':0}

def tabulateSummary(runs, printFlag = True):
    """ Function to tabulate a general summary of a run. """
    
    woodKillsReq = 1000
    villKillsReq = 1000
    castKillsReq = 1000
    
    woodBossKilled = 0
    villBossKilled = 0
    castBossKilled = 0
    
    bugbear  = 0
    werewolf = 0
    
    zombie   = 0
    ghost    = 0
    
    skeleton = 0
    vampire  = 0
    
    machine = 0
    
    # some loot/actions mess w/ the string parsing here; manually exclude em
    exclTable = ['ghost pepper', 'HOA zombie eyes', 'zombie accordion', 
                 'zombie mariachi pants', 'zombie mariachi hat', 
                 'drove some zombies out', 'ghost pencil', 'ghost shawl',
                 'drove some skeletons out', 'drove some vampires']
    
    for row in runs.split('\n'):
        if any(excl in row for excl in exclTable):
            continue
        elif ('bugbear'  in row):
            bugbear  += lineParse(row)['kills']
        elif ('werewolf' in row):
            werewolf  += lineParse(row)['kills']
        elif ('zombie'   in row):
            zombie  += lineParse(row)['kills']
        elif ('ghost'    in row):
            ghost  += lineParse(row)['kills']
        elif ('skeleton' in row):
            skeleton  += lineParse(row)['kills']
        elif ('vampire'  in row):
            vampire  += lineParse(row)['kills']
        elif ('used The Machine' in row):
            machine += 1
            
        # bosses
        elif ('  Falls-From-Sky' in row):
            woodBossKilled += lineParse(row)['kills']
        elif ('The Great Wolf of the Air ' in row):
            woodBossKilled += lineParse(row)['kills']
        elif ('  Mayor Ghost' in row):
            villBossKilled += lineParse(row)['kills']
        elif (' the Zombie Homeowners\' Association ' in row):
            villBossKilled += lineParse(row)['kills']
        elif ('  Count Drunkula ' in row):
            castBossKilled += lineParse(row)['kills']
        elif ('  The Unkillable Skeleton' in row):
            castBossKilled += lineParse(row)['kills']
    
    # was bugchecking a stupid error
    # print("{} {} | {} {} | {} {}".format(bugbear,werewolf,zombie,ghost,vampire,skeleton))
    
    woodPct = bugbear/(bugbear+werewolf+0.00001)
    villPct = zombie/(zombie+ghost+0.00001)
    castPct = vampire/(vampire+skeleton+0.00001)
    
    if woodPct>0.5: woodBoss = 'FFS ({:0.2f})'.format(woodPct)
    else:           woodBoss = 'Wolf ({:0.2f})'.format(1-woodPct)
        
    if villPct>0.5: villBoss = 'ZHoA ({:0.2f})'.format(villPct)
    else:           villBoss = 'MG ({:0.2f})'.format(1-villPct)
        
    if castPct>0.5: castBoss = 'Drunkula ({:0.2f})'.format(castPct)
    else:           castBoss = 'UKS ({:0.2f})'.format(1-castPct)
    
    if printFlag == True:   
        print('==== RUN SUMMARY ==== \n   KILLS LEFT: {}/{}/{}\n   BOSSES: {}, {}, {}\n   MACHINE: {} skills left!\n   BOSS DEFEATS: {} {} {}'.format(
        1000-(bugbear+werewolf),1000-(ghost+zombie),1000-(skeleton+vampire),woodBoss,villBoss,castBoss,3-machine,woodBossKilled, villBossKilled, castBossKilled))
    
    return '{}/{}/{} ({} skills left! -- {}, {}, {})'.format(
        1000-(bugbear+werewolf),1000-(ghost+zombie),1000-(skeleton+vampire),3-machine,woodBoss,villBoss,castBoss)
    

In [225]:
# commented out because it's quite long. but if you want to summarize everything, this is it!

# for run in runFiles:
#     print('=====================\n---- {}'.format(run))
#     tabulateSummary(open('{}{}.txt'.format(raidDir, run)).read()+"\n")

# also, while i'm commenting stuff out, 

# n = "The Dictator"
# for run in runFiles:
#     print('{} - {}'.format(run,str(tabulateKills(n, open('{}{}.txt'.format(raidDir, run)).read()+"\n"))))

In [232]:
def buildTextUpdate(numInst = 5, rDir = newRDir,desc = 'yolo yolo yolo'):
    ''' Grabs the five most recent instances & generates info '''
    
    # most recent files using some garbo processing
    fileTree = os.popen('ls -lrst "{}"'.format(rDir)).read().split('\n')[-1*(numInst+1):]
    
    infoDump = {}
    orderedL = []
    
    for fileRaw in fileTree:
        # unix system generates this in fixed width
        if len(fileRaw) > 53:
            file = fileRaw[53:]
            clan = clanMap[file.split(sep='_')[1]]
            infoDump[clan] = tabulateSummary(open('{}{}'.format(rDir, file)).read()+"\n",printFlag=False)
    
    for clan in infoDump:
        kLeft = sum([int(kills) for kills in infoDump[clan].split(' ')[0].split('/')])
        orderedL = orderedL + [(kLeft,clan)]
    
    orderedL.sort()
    
    clanUpdate = ''
    
    for x in orderedL:
        clanUpdate = '**{}:** '.format(x[1]) + infoDump[x[1]] + '\n' + clanUpdate
            
    print('** {} ASS DREAD UPDATE **\n\nhey @Dungeon Runners ! {}\n\n{}\nand your skill ladder!\n'.format(currDate,desc,clanUpdate))
    
    return infoDump

In [235]:
# just update the description and it'll generate a paste-able update! whoo!

description = "i have finally, thankfully, scripted it so that generating \
this is easier than it used to be. it's a dog's code, but it works. hooray! \
the only new thing is that it now gives the % of monsters are that boss' phylum. \
this is probably not particularly useful to most folks, but be careful w/ ones \
under 60%"

buildTextUpdate(5,newRDir, desc = description)

** 0503 ASS DREAD UPDATE **

hey @Dungeon Runners ! i have finally, thankfully, scripted it so that generating this is easier than it used to be. it's a dog's code, but it works. hooray! the only new thing is that it now gives the % of monsters are that boss' phylum. this is probably not particularly useful to most folks, but be careful w/ ones under 60%

**Piglets:** 1000/1000/1000 (3 skills left! -- Wolf (1.00), MG (1.00), UKS (1.00))
**HoE:** 781/781/781 (3 skills left! -- FFS (0.62), MG (0.57), UKS (0.58))
**KirbyClan:** 10/10/10 (3 skills left! -- FFS (0.61), MG (0.59), UKS (0.61))
**Greyhawk:** 8/10/10 (3 skills left! -- FFS (0.60), ZHoA (0.60), Drunkula (0.60))
**FactClub:** 0/0/0 (0 skills left! -- FFS (0.60), ZHoA (0.62), Drunkula (0.61))

and your skill ladder!



{'HoE': '781/781/781 (3 skills left! -- FFS (0.62), MG (0.57), UKS (0.58))',
 'KirbyClan': '10/10/10 (3 skills left! -- FFS (0.61), MG (0.59), UKS (0.61))',
 'Piglets': '1000/1000/1000 (3 skills left! -- Wolf (1.00), MG (1.00), UKS (1.00))',
 'Greyhawk': '8/10/10 (3 skills left! -- FFS (0.60), ZHoA (0.60), Drunkula (0.60))',
 'FactClub': '0/0/0 (0 skills left! -- FFS (0.60), ZHoA (0.62), Drunkula (0.61))'}